In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Train and Test

In [2]:
import pandas as pd
import gc

dirThis = '/content/drive/MyDrive/1經濟學/專題/'

trainHistory = pd.read_csv(dirThis + 'trainHistory.csv')
testHistory = pd.read_csv(dirThis + 'testHistory.csv')

Concat Split data

In [3]:
split = pd.read_csv(dirThis + 'outputSQL/sqlout18.csv', header = 0, nrows = 100000)

trans_int = split.select_dtypes(include=['int'])
converted_int = trans_int.apply(pd.to_numeric,downcast='unsigned')
trans_float = split.select_dtypes(include=['float'])
converted_float = trans_float.apply(pd.to_numeric,downcast='float')
optimized_trans = split.copy()
optimized_trans[converted_int.columns] = converted_int
optimized_trans[converted_float.columns] = converted_float
converted_obj = pd.DataFrame()

trans_obj = split.select_dtypes(include=['object']).copy()
for col in trans_obj.columns:
    num_unique_values = len(trans_obj[col].unique())
    num_total_values = len(trans_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = trans_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = trans_obj[col]
optimized_trans[converted_obj.columns] = converted_obj

dtypes = optimized_trans.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
preview = { key:value for key,value in list(column_types.items())[::] }

In [4]:
split = pd.read_csv(dirThis + 'outputSQL/sqlout1.csv', dtype = preview)

In [5]:
split

,id,chain,offer,market,offerdate,offervalue,buy_total_amount,buy_total_avgamount,buy_total_freq,buy_total_daydiff,...,buy_category_freq_180,buy_category_CT_180,buy_category_OZ_180,buy_category_LT_180,not_buy_company,not_buy_brand,not_buy_category,not_buy_company_brand,not_buy_company_category,not_buy_brand_category
0,86246,205,1208251,34,2013/4/24,2.0,52828.12,4.189715,12609,1,...,8949,1513.0,10517.0,145.0,0,0,1,0,0,0
1,15753725,17,1208251,4,2013/4/24,2.0,9406.59,5.168456,1820,4,...,1168,196.0,1903.0,22.0,0,1,1,0,0,1
2,16535563,4,1208251,1,2013/4/27,2.0,6374.61,6.299022,1012,16,...,557,96.0,728.0,0.0,0,0,1,0,0,0
3,18259179,3,1208251,2,2013/4/24,2.0,913.40,4.636548,197,10,...,104,20.0,94.0,3.0,0,0,1,0,0,0
4,21024070,15,1208251,9,2013/4/23,2.0,7622.29,3.747438,2034,1,...,802,102.0,761.0,22.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19348,130694037,192,1220503,22,2013/7/22,1.5,891.48,3.084706,289,36,...,89,11.0,92.0,0.0,0,0,0,0,0,0
19349,132113969,192,1220503,22,2013/7/19,1.5,2348.03,3.442859,682,7,...,231,40.0,265.0,0.0,0,0,1,0,0,0
19350,132663357,192,1220503,22,2013/7/5,1.5,2527.37,3.067197,824,6,...,312,40.0,334.0,0.0,1,1,0,1,0,0
19351,132806407,192,1220503,22,2013/7/18,1.5,2684.53,2.953278,909,2,...,251,19.0,234.0,1.0,0,0,0,0,0,0


In [ ]:
i = 1
total = 18
split = pd.read_csv(dirThis + 'outputSQL/sqlout1.csv', dtype = preview)
i += 1
for re in range(total-1) :
  split1 = pd.read_csv(dirThis + 'outputSQL/sqlout' + str(i) + '.csv', dtype = preview)
  print("split1.shape", split1.shape)
  i += 1
  split = pd.concat([split, split1], axis = 0, ignore_index = True)
  del split1
  gc.collect()

# split.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
print(trainHistory.shape[0]+testHistory.shape[0])
print(split.shape[0])

311541
311558


Data Preprocessing

In [ ]:
# split = pd.read_csv(dirThis + 'featureNew/use.csv')

In [ ]:
# for i in split.columns :
#     print("'"+i+"',")

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
trainData = split[split['id'].isin(trainHistory['id'])] # (159700,
testData = split[split['id'].isin(testHistory['id'])] # (150965
rep = trainHistory[['id', 'repeater']]
rept = trainHistory[['id', 'repeattrips']]
target = rep[rep['id'].isin(trainData['id'])] # (159700,
targetT = rept[rept['id'].isin(trainData['id'])] # (159700,

# ordinal encode
cod = OrdinalEncoder()
cod = cod.fit(split[['market', 'offer', 'chain']])
split[['market', 'offer', 'chain']] = cod.transform(split[['market', 'offer', 'chain']])
trainEncode = split[split['id'].isin(trainHistory['id'])] # (159700,
testEncode = split[split['id'].isin(testHistory['id'])] # (150965

# encode for target
targetEncode = target.copy()
cod = LabelEncoder()
targetEncode['repeater'] = cod.fit_transform(targetEncode['repeater'])


select = [
    'buy_total_amount',
    'buy_total_avgamount',
    'buy_total_freq',
    'buy_total_daydiff',
    'buy_total_CT',
    'buy_total_OZ',
    'buy_total_LT',
    'buy_total_amount_30',
    'buy_total_avgamount_30',
    'buy_total_freq_30',
    'buy_total_CT_30',
    'buy_total_OZ_30',
    'buy_total_LT_30',
    'buy_total_amount_60',
    'buy_total_avgamount_60',
    'buy_total_freq_60',
    'buy_total_CT_60',
    'buy_total_OZ_60',
    'buy_total_LT_60',
    'buy_total_amount_90',
    'buy_total_avgamount_90',
    'buy_total_freq_90',
    'buy_total_CT_90',
    'buy_total_OZ_90',
    'buy_total_LT_90',
    'buy_total_amount_120',
    'buy_total_avgamount_120',
    'buy_total_freq_120',
    'buy_total_CT_120',
    'buy_total_OZ_120',
    'buy_total_LT_120',
    'buy_total_amount_150',
    'buy_total_avgamount_150',
    'buy_total_freq_150',
    'buy_total_CT_150',
    'buy_total_OZ_150',
    'buy_total_LT_150',
    'buy_total_amount_180',
    'buy_total_avgamount_180',
    'buy_total_freq_180',
    'buy_total_CT_180',
    'buy_total_OZ_180',
    'buy_total_LT_180',
    'buy_company_amount',
    'buy_company_avgamount',
    'buy_company_freq',
    'buy_company_daydiff',
    'buy_company_CT',
    'buy_company_OZ',
    'buy_company_LT',
    'buy_brand_amount',
    'buy_brand_avgamount',
    'buy_brand_freq',
    'buy_brand_daydiff',
    'buy_brand_CT',
    'buy_brand_OZ',
    'buy_brand_LT',
    'buy_category_amount',
    'buy_category_avgamount',
    'buy_category_freq',
    'buy_category_daydiff',
    'buy_category_CT',
    'buy_category_OZ',
    'buy_category_LT',
    'buy_company_amount_30',
    'buy_company_avgamount_30',
    'buy_company_freq_30',
    'buy_company_CT_30',
    'buy_company_OZ_30',
    'buy_company_LT_30',
    'buy_brand_amount_30',
    'buy_brand_avgamount_30',
    'buy_brand_freq_30',
    'buy_brand_CT_30',
    'buy_brand_OZ_30',
    'buy_brand_LT_30',
    'buy_category_amount_30',
    'buy_category_avgamount_30',
    'buy_category_freq_30',
    'buy_category_CT_30',
    'buy_category_OZ_30',
    'buy_category_LT_30',
    'buy_company_amount_60',
    'buy_company_avgamount_60',
    'buy_company_freq_60',
    'buy_company_CT_60',
    'buy_company_OZ_60',
    'buy_company_LT_60',
    'buy_brand_amount_60',
    'buy_brand_avgamount_60',
    'buy_brand_freq_60',
    'buy_brand_CT_60',
    'buy_brand_OZ_60',
    'buy_brand_LT_60',
    'buy_category_amount_60',
    'buy_category_avgamount_60',
    'buy_category_freq_60',
    'buy_category_CT_60',
    'buy_category_OZ_60',
    'buy_category_LT_60',
    'buy_company_amount_90',
    'buy_company_avgamount_90',
    'buy_company_freq_90',
    'buy_company_CT_90',
    'buy_company_OZ_90',
    'buy_company_LT_90',
    'buy_brand_amount_90',
    'buy_brand_avgamount_90',
    'buy_brand_freq_90',
    'buy_brand_CT_90',
    'buy_brand_OZ_90',
    'buy_brand_LT_90',
    'buy_category_amount_90',
    'buy_category_avgamount_90',
    'buy_category_freq_90',
    'buy_category_CT_90',
    'buy_category_OZ_90',
    'buy_category_LT_90',
    'buy_company_amount_120',
    'buy_company_avgamount_120',
    'buy_company_freq_120',
    'buy_company_CT_120',
    'buy_company_OZ_120',
    'buy_company_LT_120',
    'buy_brand_amount_120',
    'buy_brand_avgamount_120',
    'buy_brand_freq_120',
    'buy_brand_CT_120',
    'buy_brand_OZ_120',
    'buy_brand_LT_120',
    'buy_category_amount_120',
    'buy_category_avgamount_120',
    'buy_category_freq_120',
    'buy_category_CT_120',
    'buy_category_OZ_120',
    'buy_category_LT_120',
    'buy_company_amount_150',
    'buy_company_avgamount_150',
    'buy_company_freq_150',
    'buy_company_CT_150',
    'buy_company_OZ_150',
    'buy_company_LT_150',
    'buy_brand_amount_150',
    'buy_brand_avgamount_150',
    'buy_brand_freq_150',
    'buy_brand_CT_150',
    'buy_brand_OZ_150',
    'buy_brand_LT_150',
    'buy_category_amount_150',
    'buy_category_avgamount_150',
    'buy_category_freq_150',
    'buy_category_CT_150',
    'buy_category_OZ_150',
    'buy_category_LT_150',
    'buy_company_amount_180',
    'buy_company_avgamount_180',
    'buy_company_freq_180',
    'buy_company_CT_180',
    'buy_company_OZ_180',
    'buy_company_LT_180',
    'buy_brand_amount_180',
    'buy_brand_avgamount_180',
    'buy_brand_freq_180',
    'buy_brand_CT_180',
    'buy_brand_OZ_180',
    'buy_brand_LT_180',
    'buy_category_amount_180',
    'buy_category_avgamount_180',
    'buy_category_freq_180',
    'buy_category_CT_180',
    'buy_category_OZ_180',
    'buy_category_LT_180'
]

trainStd = trainEncode.copy()
X = trainEncode[select]
scaler = StandardScaler()
trainStd[select] = scaler.fit_transform(X)

testStd = testEncode.copy()
X = testEncode[select]
scaler = StandardScaler()
testStd[select] = scaler.fit_transform(X)

Export Data

In [ ]:
target.to_csv(dirThis + 'featureNew/BIG/targetBIG.csv')
targetT.to_csv(dirThis + 'featureNew/BIG/targetTripsBIG.csv')
trainEncode.to_csv(dirThis + 'featureNew/BIG/trainEncodeBIG.csv')
testEncode.to_csv(dirThis + 'featureNew/BIG/testEncodeBIG.csv')
targetEncode.to_csv(dirThis + 'featureNew/BIG/targetEncodeBIG.csv')
trainStd.to_csv(dirThis + 'featureNew/BIG/trainStdBIG.csv')
testStd.to_csv(dirThis + 'featureNew/BIG/testStdBIG.csv')

In [ ]:
print(trainEncode.shape)
print(testEncode.shape)

(160062, 184)
(151496, 184)
